#  Predictive Maintenance for Manufacturing Machines

!<img src="assets/images.jpeg" alt="Workflow" width="500">

**Problem:** Unexpected machine failures lead to downtime and high costs.

**Goal:** Predict machine failures and generate actionable Failure Scheduling Reports to prevent them.




### **Overview:** Utilizing sensor data and implementing an ML pipeline to build a model that predicts mechanical failures before they occur.

* **Estimated time:** 45 minutes
* **Difficulty:** Intermediate

---

##  Operational Environment & Objective
(Visuals of Equipment / Data)

This project aims to analyze real-time operational data from a set of industrial machines. By monitoring key variables such as temperature, vibration, and rotational speed, we can identify patterns that precede mechanical failure.

**Core Objective:** To transition from reactive maintenance (repair after failure) to predictive maintenance (repair before failure).



---

##  The Problem: Cost of Unplanned Downtime
(Problem Statement)

Unplanned machine downtime is incredibly costly, leading to lost production and increased emergency repair expenses. This project directly addresses this issue.

| Normal Operation | Failure State |
| :---: | :---: |
|  |  |
| The motor operates efficiently within normal temperature and vibration ranges. | A sudden temperature spike and evident wear led to machine failure. |

---

##  Data Table Preview
(Data Table Preview)

Our dataset comprises multiple sensor readings collected over time, along with a target variable indicating whether a failure occurred.

**Sample Sensor Data:**

| Machine ID | Temp (K) | Torque (Nm) | Rotational Speed (rpm) | Failure? |
| :--- | :---: | :---: | :---: | :---: |
| M14860 | 302.6 | 60.1 | 1625 | 0 |
| M14861 | 302.7 | 42.8 | 1410 | 0 |
| M14862 | 301.9 | 47.6 | 1782 | 1 (TWF) |
| M14863 | 302.4 | 47.9 | 1667 | 0 |
| M14864 | 302.5 | 40.2 | 1768 | 0 |

> **Note:** `Failure? = 0` means normal operation, while `Failure? = 1` indicates an error. (TWF = Tool Wear Failure).

---

## Project Roadmap
(Workflow Diagram)

To achieve our goal, we will follow a standard data science workflow:



1.  **Data Collection:** Loading sensor data.
2.  **Preprocessing:** Handling missing data and feature engineering.
3.  **Modeling:** Training various models (e.g., Random Forest, XGBoost) to detect failures.
4.  **Evaluation:** Measuring model accuracy and predictive capability.
5.  **Predictions:** Using the model to forecast future maintenance schedules.

---

##  Let's Start Implementing
(Interactive Notes & Structure)

Now, let's dive into the code. The notebook is structured into the following sections for easy navigation:

1.  **[1]  Importing Libraries & Loading Data**
2.  **[2]  Exploratory Data Analysis (EDA)**
3.  **[3]  Feature Engineering & Preprocessing**
4.  **[4]  Training & Evaluating the Machine Learning Model**
5.  **[5]  Interpreting Results & Final Conclusion**

---
*(... code starts here ...)*

## Part 1: Obtaining the Data

In [ ]:
# ========== Standard Libraries ==========
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from scipy import stats

# ========== Scikit-learn ==========
from sklearn.model_selection import train_test_split, learning_curve, validation_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Metrics & Evaluation
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    RocCurveDisplay,
    PrecisionRecallDisplay,
    ConfusionMatrixDisplay,
    roc_auc_score,
    classification_report

)

# ========== Imbalanced Data ==========
from imblearn.over_sampling import SMOTE

# ========== XGBoost ==========
from xgboost import XGBClassifier

# ========== Save Model==========
import joblib


## Part 2: Scrubbing the Data

In [ ]:
df=pd.read_csv("predictive_maintenance.csv")
df.head(5)


In [ ]:
df.info()


In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
sns.heatmap(df.isnull())

## Part 3: Exploring the Data

In [ ]:

#===========================================================
# 1) TARGET DISTRIBUTION (Interactive Donut Chart)
#===========================================================

fig_target = px.pie(
    df,
    names="Target",
    hole=0.5,
    color="Target",
    color_discrete_map={0:"#1f77b4", 1:"#d62728"},
    title="Failure vs Normal Distribution"
)
fig_target.update_layout(width=500, height=400)
fig_target.show()

💡 **Practical Insight**  
This chart is a classic example of visualizing the **target variable** in a **binary classification problem**.

**Business Context:**  
The title **"Failure vs Normal Distribution"** strongly suggests that this dataset is used to **predict failures**.  

- **0 (Blue)** → "Normal" or Negative class  
- **1 (Red)** → "Failure" or Positive class (the event we want to predict)  

**Key Insight:**  
The chart shows **96.6% Normal vs 3.39% Failure**, which immediately reveals that the dataset is **highly imbalanced**. The "Failure" class is a **rare event**.

**Why this matters:**  
When training a machine learning model, this imbalance is critical. A naive model could achieve **96.6% accuracy** simply by predicting "Normal" every time, **failing completely to detect failures**.  

This chart is an essential first step, alerting the data scientist to apply **special techniques** such as:  
- **Oversampling / Undersampling**  
- **SMOTE or ADASYN**  
- Using appropriate evaluation metrics like **F1-Score**, **Precision-Recall AUC**, or **Balanced Accuracy**  

These approaches help in building a model that can **effectively detect rare failure events**, not just the majority class.


In [ ]:

# Set the visual theme for all plots
sns.set_theme(style="whitegrid")

# --- 1. Understand Your Target: Failure & Imbalance ---
print("\n--- Generating Plot 1: Target Variable Analysis ---")
plt.figure(figsize=(15, 6))

# Plot 1: Target (Failure vs. No Failure)
plt.subplot(1, 2, 1)
ax1 = sns.countplot(data=df, x='Target')
plt.title('Target Variable Distribution')
plt.xlabel('Target (0 = No Failure, 1 = Failure)')
plt.ylabel('Count')
# Add count labels
for p in ax1.patches:
    ax1.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', xytext=(0, 5), textcoords='offset points')

💡 **Practical Insight**  
This bar chart provides the **same insight** as the donut chart, but in a **more direct way**.

**Key Insight:**  
The chart highlights the **severe class imbalance** in the dataset:

- **0 (No Failure) → Blue:** 9661 instances  
- **1 (Failure) → Red:** 339 instances  

**Why this chart is useful:**  
- The **height of the bars** gives an immediate sense of quantity.  
- Exact counts are **annotated on top of the bars**, making the imbalance unmistakably clear.  
- No need to hover or read the y-axis—the difference is obvious at a glance.

**Context:**  
The command `plt.subplot(1, 2, 1)` suggests this is **part 1 of a two-part visualization**, likely paired with a donut chart or another plot to explore the target variable.


In [ ]:

# Plot 2: Failure Type (for failures only)
plt.subplot(1, 2, 2)
# We only want to plot actual failures, not the 'No Failure' category
failure_data = df[df['Target'] == 1]
ax2 = sns.countplot(data=failure_data, x='Failure Type')
plt.title('Failure Type Distribution (when Target == 1)')
plt.xlabel('Failure Type')
plt.ylabel('Count')
plt.xticks(rotation=30)
# Add count labels
for p in ax2.patches:
    ax2.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

💡 **Practical Insight**  
This chart is an excellent example of a **drill-down analysis**.

**Context:**  
The first plot showed that **failures were occurring (339 instances)**.  
This second plot answers the next logical question:  
> "What kind of failures are they?"

**Key Insight:**  
By filtering for `Target == 1`, this chart breaks down the **339 failure cases** into their specific **root causes**.  

- **Heat Dissipation Failure:** 112 instances (most frequent)  
- **Power Failure:** 95 instances (second most common)  
- **No Failure:** 9 instances (interesting, possibly a **data entry error** or a flagged failure without a specific type)  

**Business Value:**  
This analysis is highly actionable. Instead of just knowing that there are **339 failures**, a maintenance or engineering team can now focus efforts where it matters most:  

> "Focus on fixing heat dissipation and power supply issues,"  

which would address **207 out of 339 failures (~61%)**, significantly improving efficiency and impact.


In [ ]:

# --- 2. Analyze Sensor & Process Distributions ---
print("\n--- Generating Plot 2: Sensor & Process Distributions ---")
numerical_features = [
    'Air temperature [K]', 
    'Process temperature [K]', 
    'Rotational speed [rpm]', 
    'Torque [Nm]', 
    'Tool wear [min]'
]

# Create a figure with subplots (2 columns)
plt.figure(figsize=(15, 12))
plt.suptitle('Sensor & Process Feature Distributions', fontsize=16, y=1.02)

for i, col in enumerate(numerical_features):
    # Plot 1: Histogram (Distribution)
    plt.subplot(len(numerical_features), 2, 2*i + 1)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    
    # Plot 2: Box Plot (Outliers)
    plt.subplot(len(numerical_features), 2, 2*i + 2)
    sns.boxplot(x=df[col])
    plt.title(f'Box Plot of {col}')

plt.tight_layout(rect=[0, 0, 1, 1]) # Adjust layout to prevent suptitle overlap
plt.show()

💡 **Practical Insight**  
This is a **fundamental step** in Exploratory Data Analysis (EDA).  
The goal is to **understand the dataset's features** before using them for machine learning. This visual **dashboard** is extremely valuable.

**Complementary Views:**  
The code cleverly plots **two different charts for each feature**:

- **Histogram (Left):**  
  Shows the **shape of the distribution**.  
  - `'Air temperature'`, `'Process temperature'`, `'Rotational speed'` → roughly **normal (bell curve)**  
  - `'Torque'` and `'Tool wear'` → **non-normal shapes**, slightly skewed

- **Box Plot (Right):**  
  Shows the **summary and outliers**.  
  - The **box** represents the **middle 50%** of the data  
  - The **whiskers** show the rest of the data  
  - Individual dots (e.g., in `'Rotational speed [rpm]'` and `'Torque [Nm]'`) are **outliers**—points that are statistically far from the rest  

**Key Business Finding:**  
The most immediate observation is the presence of **significant outliers** in `'Rotational speed'` and `'Torque'`.  
A data scientist must investigate whether these are:

1. **Real events** (e.g., a machine spinning too fast or a sudden spike in torque) that might **predict failure**, or  
2. **Sensor errors / bad data** that should be **cleaned or removed**  

This analysis is **critical** for:  
- **Data cleaning**  
- Deciding how to **scale or normalize** these features before feeding them into a predictive model


In [ ]:
# --- 3. Find Relationships ---

# 3a. Correlation Heatmap
print("\n--- Generating Plot 3a: Correlation Heatmap ---")
plt.figure(figsize=(10, 8))
# Select only numerical columns for correlation
corr_df = df[numerical_features + ['Target']]
corr = corr_df.corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# 3b. Feature vs. Failure (Box Plots)
print("\n--- Generating Plot 3b: Feature vs. Failure (Box Plots) ---")
plt.figure(figsize=(18, 10))
plt.suptitle('Numerical Features vs. Failure (Target)', fontsize=16, y=1.02)
for i, col in enumerate(numerical_features):
    plt.subplot(2, 3, i + 1) # Arrange in a 2x3 grid
    sns.boxplot(data=df, x='Target', y=col)
    plt.title(f'{col}')
    plt.xlabel('Target (0 = No Failure, 1 = Failure)')

plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

💡 **Practical Insight**  
This is one of the **most powerful and common visualizations** in predictive maintenance.  
It directly answers the question:  
> "Do my sensor readings look different right before a failure?"

**How to read it:**  
By comparing the **"0" (No Failure)** box with the **"1" (Failure)** box for each feature, we can identify signals that predict a problem.

**Strongest Predictor:**  
- **Tool wear [min]:** The "Failure" (1) box is almost entirely **above** the "No Failure" (0) box.  
  - Failures almost always occur when **tool wear is high** (e.g., > 200 min).

**Strong Predictors:**  
- **Torque [Nm]:** Failures (1) are clearly associated with **higher torque** values.  
- **Rotational speed [rpm]:** Failures (1) are associated with **lower rotational speeds**.

**Weaker Predictors:**  
- **Air temperature [K]** and **Process temperature [K]:** Failures (1) tend to occur at **slightly higher temperatures**, but there is significant **overlap** with the "No Failure" (0) boxes, making them less reliable individually.

**Business Value:**  
This chart enables a shift from **reactive maintenance to predictive maintenance**.  
- A machine learning model could **learn these patterns**.  
- Even without a model, engineers could set up a simple alert:  
> "If tool wear > 200 min **AND** torque > 60 Nm **AND** rotational speed < 1400 rpm, flag the machine for inspection."


In [ ]:

# 3c. Torque vs. Rotational Speed (Scatter Plot)
print("\n--- Generating Plot 3c: Torque vs. Rotational Speed ---")
plt.figure(figsize=(12, 8))
# Use a subset if the dataset is huge, but 10,000 is fine
sns.scatterplot(
    data=df.sample(n=min(5000, len(df))), # Sample to avoid overplotting if N is very large
    x='Rotational speed [rpm]', 
    y='Torque [Nm]', 
    hue='Target', 
    palette={0: '#0072B2', 1: '#D55E00'}, # Color-blind friendly
    alpha=0.5,
    style='Target'
)
plt.title('Torque vs. Rotational Speed (Colored by Failure)')
plt.legend(title='Target', labels=['1 - Failure', '0 - No Failure'])
plt.show()

💡 **Practical Insight**  
This plot provides a **"map" of the machine's operating behavior** and its **"danger zones."**

**Main Relationship:**  
- There is a **strong negative correlation** between speed and torque.  
- As **speed increases**, **torque decreases**.  
- This is typical for many motors (**Power ≈ Torque × Speed**).  
  - For relatively constant power, if one goes up, the other must come down.

**Key Finding (Failure Clusters):**  
The failures (orange 'x's) are **not random**; they are concentrated in **two distinct regions**:

1. **Low Speed / High Torque:**  
   - Cluster in top-left (**speed < 1750, torque > 40**)  
   - Represents a **"high-strain" / "lugging"** condition

2. **High Speed / Low Torque:**  
   - Cluster in bottom-right (**speed > 2250, torque < 20**)  
   - Represents a **"high-spin" / "low-load"** condition

**Business Value:**  
- Failure is not simply "too much torque" or "too much speed."  
- It's the **combination of variables** that predicts failure.  
- This plot **defines the safe operating envelope** (dense blue area) and highlights **high-risk "danger zones"**.  
- A **predictive maintenance model** could learn to flag these zones to prevent failures.


In [ ]:


# --- 4. Analyze Categorical Data (Product Type) ---
print("\n--- Generating Plot 4: Categorical Data Analysis ---")
plt.figure(figsize=(15, 6))

# Plot 1: Absolute Counts
plt.subplot(1, 2, 1)
sns.countplot(data=df, x='Type', hue='Target', order=['L', 'M', 'H'])
plt.title('Failure Counts by Product Type')
plt.legend(title='Target', labels=['0 - No Failure', '1 - Failure'])

# Plot 2: Normalized (Failure Rate)
plt.subplot(1, 2, 2)
# Calculate failure rate per type
failure_rate = df.groupby('Type')['Target'].mean().reset_index()
sns.barplot(data=failure_rate, x='Type', y='Target', order=['L', 'M', 'H'], palette='viridis')
plt.title('Average Failure Rate by Product Type')
plt.ylabel('Failure Rate (Proportion)')

plt.tight_layout()
plt.show()


💡 **Practical Insight**  
This is a **perfect example of why normalization is critical**.

**Plot 1 (Absolute Counts):**  
- Misleading at first glance.  
- Shows that **"L" (Low) type products** have the most failures (orange bar is tallest).  
- A manager might wrongly conclude:  
> "L type products are the biggest problem."  
- The reason the bar is tallest is simply that **"L" is the most common product type** in the dataset (blue bar is also tallest).

**Plot 2 (Failure Rate):**  
- Tells the **real story**.  
- Answers the question:  
> "Given a product of type L, M, or H, what is its probability of failing?"  
- Shows that **"L" type products have the highest failure rate (~3.9%)**, while **"H" type products**, despite having some failures, are actually the **most reliable**.

**Business Value:**  
This analysis correctly identifies that the **"L" product line is proportionally the least reliable**, even though it is the most common.  
- Guides the engineering team to **investigate why L type fails more often**, rather than just focusing on it due to total failure counts.


## 4. Modeling the Data

In [ ]:
from scipy import stats
import numpy as np

numeric_cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]',
                'Torque [Nm]', 'Tool wear [min]']

z_scores = np.abs(stats.zscore(df[numeric_cols]))

outliers = df[(z_scores > 3).any(axis=1)]

print(outliers.shape)


In [ ]:

df = df.drop(["UDI","Product ID"] , axis=1)
dummies = pd.get_dummies(df["Failure Type"], drop_first=True)
df = pd.concat([df.drop("Failure Type", axis=1), dummies], axis=1)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, roc_auc_score, f1_score, 
    precision_score, recall_score, roc_curve
)
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,
    ExtraTreesClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# --- Configuration & Setup ---

# !! ASSUMPTION: 'df' is already loaded as a pandas DataFrame
# Example:
# df = pd.read_csv('your_data.csv') 

# numeric & categorical
num_cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]',
            "Torque [Nm]", "Tool wear [min]"]
categ_cols = ['Type']


# --- Preprocessing Pipeline ---

# ColumnTransformer
all_pipeline = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols),

    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
    ]), categ_cols),
], remainder='drop') 


# --- Data Splitting ---
try:
    y = df["Target"]
    x = df.drop(["Target"], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=0.3, random_state=42, stratify=y
    )

    # --- Apply Preprocessing ---
    X_train_final = all_pipeline.fit_transform(X_train)
    X_test_final = all_pipeline.transform(X_test)

    print(f"Training data shape: {X_train_final.shape}")
    print(f"Test data shape: {X_test_final.shape}")

    # ============================
    #  Dynamic Class Weight Calculation
    # ============================
    try:
        counts = np.bincount(y_train)
        scale_pos_weight = counts[0] / counts[1]
        print(f"Calculated scale_pos_weight for imbalance: {scale_pos_weight:.2f}")
    except (IndexError, ZeroDivisionError) as e:
        print(f"Warning: Could not calculate scale_pos_weight ({e}). Defaulting to 1.")
        scale_pos_weight = 1
        

    # ============================
    #  MODELS LIST
    # ============================
    models = {
        "LogisticRegression": LogisticRegression(
            max_iter=500, class_weight="balanced", random_state=42),

        "RandomForest": RandomForestClassifier(
            n_estimators=200, random_state=42, class_weight="balanced"),

        "ExtraTrees": ExtraTreesClassifier(
            n_estimators=300, random_state=42, class_weight="balanced"),
        
        "GradientBoosting": GradientBoostingClassifier(
            random_state=42), 

        "AdaBoost": AdaBoostClassifier(
            n_estimators=200, random_state=42), 

        "XGBoost": XGBClassifier(
            eval_metric="logloss", random_state=42,
            scale_pos_weight=scale_pos_weight), 

        "LightGBM": LGBMClassifier(
            class_weight='balanced', random_state=42, verbose=-1), 

        "CatBoost": CatBoostClassifier(
            iterations=1000, 
            depth=8,
            learning_rate=0.03,
            l2_leaf_reg=5,
            loss_function='Logloss',
            eval_metric='AUC',
            scale_pos_weight=scale_pos_weight, 
            random_state=42,
            verbose=0
        )
    }


    # ============================
    #  Train & Evaluation
    # ============================
    
    # List to store results for DataFrame
    results_list = []
    
    # List to store data for ROC plot
    roc_plot_data = []

    print("\n--- Starting Model Training & Evaluation ---")

    for name, model in models.items():
        print(f"Fitting {name}...")
        try:
            # Train model
            model.fit(X_train_final, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test_final)
            y_pred_proba = model.predict_proba(X_test_final)[:, 1] # Proba for class 1

            # Get metrics
            auc = roc_auc_score(y_test, y_pred_proba)
            f1 = f1_score(y_test, y_pred, average='macro')
            recall = recall_score(y_test, y_pred, average='macro')
            precision = precision_score(y_test, y_pred, average='macro')
            
            # Store results for DataFrame
            results_list.append({
                "Model": name,
                "AUC": auc,
                "F1 (Macro)": f1,
                "Recall (Macro)": recall,
                "Precision (Macro)": precision
            })
            
            # NEW: Store data for ROC Plot
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_plot_data.append({
                'name': name,
                'fpr': fpr,
                'tpr': tpr,
                'auc': auc
            })

            # Print classification report
            print(f"\n====== {name} Report ======")
            print(classification_report(y_test, y_pred))
            print(f"AUC: {auc:.4f}")
            print("-" * 30)

        except Exception as e:
            print(f"\n!!!!!! FAILED to train or evaluate {name} !!!!!!")
            print(f"Error: {e}")
            print("-" * 30)

    # ============================
    #  Final Summary Report (Table)
    # ============================
    results_df = pd.DataFrame() # Initialize empty
    if results_list:
        print("\n\n--- Model Performance Summary ---")
        results_df = pd.DataFrame(results_list)
        results_df_sorted = results_df.sort_values(by="AUC", ascending=False)
        print(results_df_sorted.to_string(index=False))
    else:
        print("No models were successfully trained.")

    
    # ======================================================
    #  NEW: 📈 Model Visualization
    # ======================================================
    
    if not results_df.empty:
        
        # --- Plot 1: ROC-AUC Curves ---
        print("\nGenerating ROC-AUC Curves Plot...")
        plt.figure(figsize=(12, 9))
        
        # Plot the "chance" line
        plt.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)')
        
        # Plot each model's ROC curve
        for item in roc_plot_data:
            plt.plot(item['fpr'], item['tpr'], 
                     label=f"{item['name']} (AUC = {item['auc']:.4f})")
        
        plt.xlabel('False Positive Rate (FPR)')
        plt.ylabel('True Positive Rate (TPR)')
        plt.title('Receiver Operating Characteristic (ROC) Curves', fontsize=16)
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        
        # --- Plot 2: Performance Metrics Comparison (Bar Charts) ---
        print("Generating Performance Metrics Comparison Plot...")
        
        # Melt the DataFrame to long format for easier plotting with Seaborn
        df_melted = results_df.melt(id_vars='Model', var_name='Metric', value_name='Score')
        
        # Create a grid of bar plots (FacetGrid)
        g = sns.catplot(
            data=df_melted,
            x='Score',
            y='Model',
            col='Metric',
            kind='bar',
            col_wrap=2,       # Wrap after 2 columns
            height=5,
            aspect=1.2,
            sharex=False,     # Each plot has its own x-axis scale
            palette='viridis'
        )
        
        # Set titles and layout
        g.set_titles("{col_name}", size=14)
        g.fig.suptitle('Model Performance Metrics Comparison', y=1.03, size=18)
        plt.tight_layout()
        plt.show()

    else:
        print("\nSkipping visualization as no models were successfully trained.")


except NameError:
    print("\nError: The DataFrame 'df' is not defined.")
    print("Please load your data into a DataFrame named 'df' before running this script.")
except KeyError as e:
    print(f"\nError: A required column {e} was not found in the DataFrame.")
    print("Please check your column names ('Target', num_cols, categ_cols).")

💡 **Practical Insight – Model Training & Evaluation**  

This section presents the **performance of multiple classification models** on the predictive maintenance dataset.  

**Context:**  
- The dataset is highly imbalanced (failures are rare).  
- Multiple models were trained with preprocessing pipelines including **imputation, scaling, and one-hot encoding**.  
- Metrics considered: **AUC**, **F1 (Macro)**, **Recall (Macro)**, **Precision (Macro)**.  

---

### **1️⃣ Key Takeaways from Metrics Table**

| Model | AUC | F1 (Macro) | Recall (Macro) | Precision (Macro) |
|-------|-----|------------|----------------|------------------|
| XGBoost | 0.9781 | 0.872 | 0.882 | 0.862 |
| LightGBM | 0.9778 | 0.841 | 0.875 | 0.813 |
| GradientBoosting | 0.9774 | 0.855 | 0.798 | 0.942 |
| CatBoost | 0.9766 | 0.850 | 0.862 | 0.839 |
| RandomForest | 0.9670 | 0.772 | 0.705 | 0.918 |
| ExtraTrees | 0.9579 | 0.723 | 0.652 | 0.958 |
| AdaBoost | 0.9527 | 0.726 | 0.687 | 0.789 |
| LogisticRegression | 0.8885 | 0.574 | 0.813 | 0.567 |

**Insights:**  
- **XGBoost, LightGBM, GradientBoosting, CatBoost** are the top-performing models.  
- Logistic Regression performs the worst on **F1 and AUC**, likely due to the **class imbalance**.  
- Ensemble methods (boosting) consistently outperform bagging methods (RandomForest, ExtraTrees) on AUC.

---

### **2️⃣ ROC-AUC Curves**

- The **ROC-AUC curve** visualizes the trade-off between **True Positive Rate** and **False Positive Rate** for each model.  
- All top models achieve **AUC > 0.97**, indicating **excellent discrimination** between failures and normal cases.  
- The diagonal line represents **random chance (AUC = 0.5)** for comparison.

---

### **3️⃣ Performance Metrics Comparison (Bar Charts)**

- Bar charts provide a **visual comparison** of all models across **F1, Recall, and Precision**.  
- Enables a quick identification of **high-precision vs high-recall models**, depending on business priorities:  
  - For predictive maintenance, **Recall** might be prioritized to **catch as many failures as possible**, even at the cost of some false positives.  
  - Precision is also important to **avoid unnecessary interventions**.  

---

✅ **Business Value:**  
- These results guide **model selection** for deployment in predictive maintenance.  
- Top models can be further tuned or used in **ensemble strategies** to maximize reliability.  
- Visualizations (ROC + Bar Charts) provide **intuitive insights for stakeholders** beyond numeric metrics.


### 🔗 Let's Connect

[![LinkedIn](https://cdn-icons-png.flaticon.com/512/174/174857.png)](https://www.linkedin.com/in/ahmed-hamdy-4569a8360/)


If you'd like to discuss projects or data science topics, connect on LinkedIn

> If anybody would like to discuss any other projects or just have a chat about data science topics, I'll be more than happy to connect with you on LinkedIn: https:[/ahmed hamdy/](https://www.linkedin.com/in/ahmed-hamdy-4569a8360/)

This notebook will always be a work in progress. Please leave any comments about further improvements to the notebook! Any feedback or constructive criticism is greatly appreciated. Thank you guys!